# Elevation Data Accuracy Assessment
This notebook creates a pipeline which determines both the most accurate interpolation method for a series of weather station locations within Minnesota and the accuracy of the point data through comparison of known elevation data. The data is then stored in a file geodatabase and moved to an .sde database for creating real-time web maps in ArcGIS Online.

### Import All Required Libraries

In [1]:
# Library for data manipulation and analysis
import pandas as pd

# Library for working with ArcGIS tools and functionalities
import arcpy
import arcgis

# Libraries for working with PostgreSQL
import psycopg2
from psycopg2 import sql

# Library for handling JSON data
import json

# Library for making HTTP requests and interacting with web services
import requests

# Library for interacting with the operating system, managing file paths, and executing system commands
import os

# Library for handling warnings generated during code execution
import warnings

# Library for displaying images in Jupyter Notebooks
from IPython.display import Image

# Library for generating random numbers
import random

# Library for handling zip files and working with IO streams
import zipfile
import io

# Library for working with dates
from datetime import date

In [2]:
file_path = os.path.dirname(arcpy.mp.ArcGISProject('CURRENT').filePath)

# Change the current working directory to the extracted directory path
os.chdir(file_path)

# Set the workspace environment to the extracted directory path
arcpy.env.workspace = file_path

# Define a spatial reference with ID 26915 (likely UTM zone 15N)
spatial_ref = arcpy.SpatialReference(26915)

# Establish variables for project and map
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]

### Path to FGDB And SDE

In [3]:
# Establish SDE Connection via PGAdmin & Catalog Pane in ArcGIS Pro
sde = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\PostgreSQL-35-gis5572(postgres).sde"

#path to local database
local_gdb = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"

In [14]:
# Create database connection
dbname = 'PostgreSQL-35-gis5572(postgres).sde'
platform = 'POSTGRESQL'
user = 'postgres'
password = '*****'
instance = '35.238.64.215'
port = '5432'
auth = 'DATABASE_AUTH'
save = 'SAVE_USERNAME'
db = 'gis5572'

arcpy.management.CreateDatabaseConnection(
    out_folder_path = file_path,
    out_name = dbname,
    database_platform = platform,
    instance = instance,
    account_authentication = auth,
    username = user,
    password = password,
    save_user_pass = save,
    database = db
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\PostgreSQL-35-gis5572(postgres).sde'>

### Grount Truth DEM

In [7]:
# Define a function to pull data from the Minnesota Geospatial Commons
def mn_geo_pull_and_unzip(url, directory, GDB_or_SHP):

    # Get GeoJSON from MN Geospatial Commons
    api = requests.get(url)
    json = api.json()
    
    # Use second list in the 'resources' key if data is in file geodatabase format
    if GDB_or_SHP == 'GDB':
        zip_link = requests.get(json['result']['resources'][1]['url'])
    
    # Use first list in the 'resources' key if data is in shapefile format
    if GDB_or_SHP == 'SHP':
        zip_link = requests.get(json['result']['resources'][2]['url'])
    
    # Get zipfile and extract
    z_file = zipfile.ZipFile(io.BytesIO(zip_link.content))
    z_file.extractall(directory)

In [27]:
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=elev-30m-digital-elevation-model',
                      r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3','GDB')

In [31]:
m.addDataFromPath(r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m')

In [34]:
# Reproject to perform resampling and converting to points

DEM_BD = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\elev_30m_digital_elevation_model.gdb'

# Resample to limit the number of output points
arcpy.management.Resample(
    in_raster = os.path.join(DEM_BD,'digital_elevation_model_30m'),
    out_raster = os.path.join(DEM_BD,'digital_elevation_model_3km'),
    cell_size = 3000 # About 3 km in vertical degrees
)

# Establish extent as the size of the ground truth DEM
arcpy.env.extent = os.path.join(DEM_BD,'digital_elevation_model_3km')

### Pulling Data from SDE to FGDB Data 

In [6]:
# Define the SDE feature class path
sde_feature_class = "gis5572.postgres.mndem_points"

# Define the local feature class path
local_feature_class = os.path.join(local_gdb, "mndem_points")

# Process: Copy Features
arcpy.CopyFeatures_management(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\PostgreSQL-35-gis5572(postgres).sde\gis5572.postgres.mndem_points", local_feature_class)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\mndem_points'>

### Create Random Sample from QAQC'd Data
Sampling 50% of the data randomly is beneficial because it reduces computational burden, maintains key dataset characteristics, and saves costs od saving and processing data. It ensures representative insights while mitigating bias, making analysis more efficient and reliable. 

In [39]:
# Set up your variables
file_gdb = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb'
# Define the well-known ID (WKID) of the spatial reference (e.g., EPSG code)
spatial_ref = arcpy.SpatialReference(4326) 
# Sample percent for random sampling
sample_percent = 50  

# Step 1: Create the Random_Sample_50_dem feature class
arcpy.management.CreateFeatureclass(
    out_path=file_gdb,
    out_name='Random_Sample_50_dem',
    geometry_type='POINT',
    spatial_reference=spatial_ref
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\Random_Sample_50_dem'>

In [40]:
# Step 2: Add fields for geometry, elevation, easting, and northing
arcpy.management.AddField(
    in_table='Random_Sample_50_dem',
    field_name='geometry',
    field_type='TEXT'
)

arcpy.management.AddField(
    in_table='Random_Sample_50_dem',
    field_name='elevation',
    field_type='DOUBLE'
)

arcpy.management.AddField(
    in_table='Random_Sample_50_dem',
    field_name='X',
    field_type='DOUBLE'
)

arcpy.management.AddField(
    in_table='Random_Sample_50_dem',
    field_name='Y',
    field_type='DOUBLE'
)

# Step 3: Get total count of features in mndem_points
total_count = arcpy.management.GetCount(r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points')[0]

# Calculate the number of features to select
sample_count = int((sample_percent / 100) * int(total_count))

# Step 4: Generate a list of random OBJECTIDs
random_objectids = list(range(1, int(total_count) + 1))
random.shuffle(random_objectids)
random_objectids = random_objectids[:sample_count]

In [41]:
# Step 5: Use the random OBJECTIDs to select features from mndem_points
where_clause = "OBJECTID IN ({})".format(','.join(map(str, random_objectids)))

arcpy.analysis.Select(
    in_features=r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points',
    out_feature_class=r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\Random_Sample_50_dem',
    where_clause=where_clause
)

# Step 6: Calculate the geometry attributes for easting and northing
arcpy.management.CalculateGeometryAttributes(
    in_features='Random_Sample_50_dem',
    geometry_property=[['easting', 'POINT_X'], ['northing', 'POINT_Y']],
    coordinate_system=spatial_ref
)

print("Random sample created successfully.")

Random sample created successfully.


### Quality Assesment of Each Model

In [10]:
# Perform exploratory interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features=r"DEM\Random_Sample_50_dem",
    value_field="grid_code",
    out_cv_table=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\stats_table_dem",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;EBK",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\stats_table_dem'>

### Interpolating the Weather Data using 3 Techniques

In [6]:
# KRINGING - ORDINARY

with arcpy.EnvManager(scratchWorkspace=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"):
    mndem_KrigOrd = arcpy.sa.Kriging(
        in_point_features=r"DEM\Random_Sample_50_dem",
        z_field="grid_code",
        kriging_model="LinearDrift 3000 # # #",
        cell_size=3000,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    mndem_KrigOrd.save(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigUni")

In [8]:
# KRINGING - UNIVERSAL

with arcpy.EnvManager(scratchWorkspace=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"):
    mndem_KrigUni = arcpy.sa.Kriging(
        in_point_features=r"DEM\Random_Sample_50_dem",
        z_field="grid_code",
        kriging_model="LinearDrift 3000 # # #",
        cell_size=3000,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    mndem_KrigUni.save(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigUni")

In [ ]:
# EMPIRICAL BAYESIAN KRIGING

arcpy.ga.EmpiricalBayesianKriging(
    in_features=r"DEM\Random_Sample_50_dem",
    z_field="grid_code",
    out_ga_layer=None,
    out_raster=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_EBK",
    cell_size=3000,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=212727.819055243 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

### Converting Interpolated Raster into Points And Exporting to SDE

In [4]:
# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

In [5]:
# Paths and parameters for each raster
raster_paths = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_EBK',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigOrd',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigUni'
]
output_downsampled_rasters = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_demEBK.tif',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_demKrigOrd.tif',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_demKrigUni.tif'
]
output_points = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\EBK_dempoints.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigOrd_dempoints.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigUni_dempoints.shp'
]
output_sde_connection = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\ArcPro\Lab2\ArcII_Lab2\PostgreSQL-35-gis5572(postgres).sde'

output_sde_feature_classes = [
    'EBK_dempoints',
    'KrigOrd_dempoints',
    'KrigUni_dempoints'
]

In [6]:
# Loop through each raster
for i in range(len(raster_paths)):
    # Step 1: Downsample Raster
    downsample_raster(raster_paths[i], output_downsampled_rasters[i], "15000")

    # Step 2: Convert Raster to Points
    raster_to_points(output_downsampled_rasters[i], output_points[i])

    # Step 3: Upload Points to SDE
    upload_points_to_sde(output_points[i], output_sde_connection, output_sde_feature_classes[i])

print("The entire 3 step process is completed.")

The entire 3 step process is completed.


### Quality Assessment

In [21]:
def accuracy_analysis(sampled_interp_layer, complete_interp_layer, output_fc):
    print(f'Performing accuracy analysis for "{sampled_interp_layer}" and "{complete_interp_layer}"...')
    
    # Check if input layers exist
    if not arcpy.Exists(sampled_interp_layer):
        print(f"Error: {sampled_interp_layer} does not exist.")
        return
    if not arcpy.Exists(complete_interp_layer):
        print(f"Error: {complete_interp_layer} does not exist.")
        return
    
    # Convert raster to points for each interpolation layer
    for layer in [sampled_interp_layer, complete_interp_layer]:
        try:
            # Check if the layer is a raster dataset
            desc = arcpy.Describe(layer)
            if desc.dataType == "RasterDataset":
                arcpy.conversion.RasterToPoint(
                    in_raster=layer,
                    out_point_features=os.path.join(arcpy.env.scratchGDB, os.path.basename(layer) + '_points')
                )
            elif desc.dataType == "FeatureClass":
                arcpy.management.CopyFeatures(
                    in_features=layer,
                    out_feature_class=os.path.join(arcpy.env.scratchGDB, os.path.basename(layer) + '_points')
                )
            else:
                print(f"Error: {layer} is not a valid data type.")
                return
        except arcpy.ExecuteError as e:
            print(f"Failed to convert {layer} to points: {e}")
            return
        
    # Spatially join the point features
    try:
        arcpy.analysis.SpatialJoin(
            target_features=os.path.join(arcpy.env.scratchGDB, os.path.basename(sampled_interp_layer) + '_points'),
            join_features=os.path.join(arcpy.env.scratchGDB, os.path.basename(complete_interp_layer) + '_points'),
            out_feature_class=os.path.join(arcpy.env.scratchGDB, output_fc),
            match_option='CLOSEST'
        )
    except arcpy.ExecuteError as e:
        print(f"Failed to perform spatial join: {e}")
        return
    
    # Add field for elevation difference
    try:
        arcpy.management.AddField(
            in_table=os.path.join(arcpy.env.scratchGDB, output_fc),
            field_name='elev_difference',
            field_type='FLOAT'
        )
    except arcpy.ExecuteError as e:
        print(f"Failed to add field 'elev_difference': {e}")
        return
    
    # Calculate elevation difference
    try:
        arcpy.management.CalculateField(
            in_table=os.path.join(arcpy.env.scratchGDB, output_fc),
            field='elev_difference',
            expression='abs(!grid_code! - !grid_code_1!)'
        )
    except arcpy.ExecuteError as e:
        print(f"Failed to calculate elevation difference: {e}")
        return
    
    # Change field names
    try:
        arcpy.management.AlterField(
            in_table=os.path.join(arcpy.env.scratchGDB, output_fc),
            field='grid_code',
            new_field_name='sampled_elev',
            new_field_alias='sampled_elev'
        )
        arcpy.management.AlterField(
            in_table=os.path.join(arcpy.env.scratchGDB, output_fc),
            field='grid_code_1',
            new_field_name='actual_elev',
            new_field_alias='actual_elev'
        )
    except arcpy.ExecuteError as e:
        print(f"Failed to alter field names: {e}")
        return
    
    # Delete unnecessary fields
    try:
        arcpy.management.DeleteField(
            in_table=os.path.join(arcpy.env.scratchGDB, output_fc),
            drop_field=['Join_Count', 'TARGET_FID', 'pointid_1']
        )
    except arcpy.ExecuteError as e:
        print(f"Failed to delete fields: {e}")
        return
    
    print('Accuracy analysis completed successfully.')

In [48]:
accuracy_analysis(
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigOrd',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points',
    'elev_ord_accuracy'
)

Performing accuracy analysis for "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigOrd" and "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points"...
The average elevation difference is: 703.273936805872


703.273936805872

In [34]:
accuracy_analysis(
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigUni',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points',
    'elev_uni_accuracy'
)

Performing accuracy analysis for "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_KrigUni" and "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points"...
The average elevation difference is: 709.0692428169984


709.0692428169984

In [47]:
accuracy_analysis(
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_EBK',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points',
    'elev_ebk_accuracy'
)

Performing accuracy analysis for "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_EBK" and "F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mndem_points"...
The average elevation difference is: 721.3599622664711


721.3599622664711

### Exporting Ground Truth and Elev Difference to SDE

In [55]:
# Destination path in the local geodatabase
output_feature = f"{local_gdb}\\elev_ord_accuracy"

elev_ord_accuracy successfully copied to F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb


In [5]:
source_gdb = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\scratch.gdb"

# Set the workspace environment
arcpy.env.workspace = source_gdb

# Copy the feature to the local geodatabase
output_feature = arcpy.CopyFeatures_management("mndem_points_points", f"{local_gdb}\\mndem_points_points")

Feature copied successfully!


In [56]:
points_list = ['elev_ord_accuracy']

for points in points_list:

    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features = points,
        Output_Geodatabase = sde
    )

In [6]:
points_list = ['mndem_points_points']

for points in points_list:

    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features = points,
        Output_Geodatabase = sde
    )